# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [43]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [44]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head(-1)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,16.22,75,46,12.98,SH,1707012125
1,1,bethel,41.3712,-73.4140,1.68,63,10,3.33,US,1707012037
2,2,vadso,70.0744,29.7487,-1.91,80,100,5.14,NO,1707012125
3,3,akureyri,65.6835,-18.0878,-6.04,93,100,1.03,IS,1707012125
4,4,ashgabat,37.9500,58.3833,0.92,86,0,2.06,TM,1707011944
...,...,...,...,...,...,...,...,...,...,...
559,559,runaway bay,18.4596,-77.3357,23.88,75,39,0.58,JM,1707012219
560,560,marsh harbour,26.5412,-77.0636,21.19,65,23,2.42,BS,1707012219
561,561,letlhakeng,-24.0944,25.0298,21.32,46,25,2.06,BW,1707012219
562,562,ola,59.5833,151.2833,-17.41,76,88,5.03,RU,1707012219


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [45]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    size="Humidity",
    scale=1,
    color="City"
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [46]:
# Narrow down cities that fit criteria and drop any results with null values
nice_temp_cities = city_data_df[(city_data_df["Max Temp"] >= 22) & (city_data_df["Max Temp"] <= 27) &
                                (city_data_df["Wind Speed"] <4.5) & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
nice_temp_cities_drop = city_data_df.dropna()

# Display sample data
nice_temp_cities.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
43,43,port elizabeth,-33.9180,25.5701,22.83,95,0,3.09,ZA,1707012133
122,122,kerikeri,-35.2268,173.9474,23.69,40,0,3.16,NZ,1707012141
198,198,campo grande,-20.4428,-54.6464,24.75,78,0,3.60,BR,1707011874
207,207,lazaro cardenas,17.9583,-102.2000,23.53,73,0,2.82,MX,1707012162
358,358,remire-montjoly,4.9167,-52.2667,26.02,94,0,1.03,GF,1707012183


### Step 3: Create a new DataFrame called `hotel_df`.

In [47]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = nice_temp_cities[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
43,port elizabeth,ZA,-33.9180,25.5701,95,
122,kerikeri,NZ,-35.2268,173.9474,40,
198,campo grande,BR,-20.4428,-54.6464,78,
207,lazaro cardenas,MX,17.9583,-102.2000,73,
358,remire-montjoly,GF,4.9167,-52.2667,94,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [48]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port elizabeth - nearest hotel: Waterford Hotel
kerikeri - nearest hotel: Avalon Resort
campo grande - nearest hotel: Hotel Metropolitan
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
remire-montjoly - nearest hotel: Complexe Belova
la toma - nearest hotel: No hotel found
whakatane - nearest hotel: Whakatane Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
43,port elizabeth,ZA,-33.9180,25.5701,95,Waterford Hotel
122,kerikeri,NZ,-35.2268,173.9474,40,Avalon Resort
198,campo grande,BR,-20.4428,-54.6464,78,Hotel Metropolitan
207,lazaro cardenas,MX,17.9583,-102.2000,73,Hotel Sol del Pacífico
358,remire-montjoly,GF,4.9167,-52.2667,94,Complexe Belova
401,la toma,AR,-33.0526,-65.6238,39,No hotel found
509,whakatane,NZ,-37.9585,176.9854,60,Whakatane Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [49]:
%%capture --no-display

# Configure the map plot
hover_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    size="Humidity",
    scale=1,
    color="City",
    hover_cols=["City", "Country", "Hotel Name"]
)

# Display the map
hover_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)